In [ ]:
%matplotlib inline
import torch
import os,sys
import numpy as np
import torchvision.io as io
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
#from torchvision import transforms 
from torch.utils.data import Dataset, random_split, DataLoader
%load_ext autoreload
%autoreload 2

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
if "google.colab" in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True, use_metadata_server=False)

In [ ]:
def import_file(url, filename):
  import requests
  r = requests.get(url)

  with open(filename, 'w') as f:
        f.write(r.text)

if "google.colab" in sys.modules:
    # Save datagenerators as file to colab working directory
    # If you are using GitHub, make sure you get the "Raw" version of the code
    url_u_net = 'https://raw.githubusercontent.com/karimassi/road-segmentation/u_net/src/models/u_net.py?token=AJVSYDHQIQ5KJXR6G5EH6NS7275YE'
    import_file(url_u_net, 'u_net.py')

    url_u_net_paper = 'https://raw.githubusercontent.com/karimassi/road-segmentation/u_net/src/models/u_net_paper.py?token=AJVSYDDDE2ZOO2E5RKHSDNK7275ZI'
    import_file(url_u_net_paper, 'u_net_paper.py')

    url_training = 'https://raw.githubusercontent.com/karimassi/road-segmentation/u_net/src/training.py?token=AJVSYDBUQJXHKKCMQZTOC3272752S'
    import_file(url_training, 'training.py')

    url_image_mask_dataset = "https://raw.githubusercontent.com/karimassi/road-segmentation/u_net/src/image_mask_dataset.py?token=AJVSYDEYVNI34EBZZW6QMPK72753U"
    import_file(url_image_mask_dataset, 'image_mask_dataset.py')

    url_losses = "https://raw.githubusercontent.com/karimassi/road-segmentation/u_net/src/losses.py?token=AJVSYDGGYYFBALHBYGZQM4S727546"
    import_file(url_losses, 'losses.py')

    url_mask_to_submission = "https://raw.githubusercontent.com/karimassi/road-segmentation/main/src/scripts/mask_to_submission.py?token=AH35XE7VAZF5BT3TVQTODU273DBZU"
    import_file(url_mask_to_submission, 'mask_to_submission.py')
        
    import u_net
    import u_net_paper 
    import training
    import image_mask_dataset
    import losses
    from mask_to_submission import masks_to_submission
else:
    from src import u_net
    from src import u_net_paper
    from src import training
    from src import image_mask_dataset
    from src import losses

In [ ]:
# load data
from image_mask_dataset import ImageMaskDataset

root_dir = "/content/drive/Shareddrives/road-segmentation/data/training/"
image_dir = root_dir + "images/"
gt_dir = root_dir + "groundtruth/"

# TODO change implementation of this dataset
dataset = ImageMaskDataset(image_dir, gt_dir, 0)

# apply transformations
rotations = [5, 10, 15, 27, 32, 44, 50, 59, 63, 70, 86, 90, 100, 120, -30, -45, -90]
for angle in rotations:
    dataset += ImageMaskDataset(image_dir, gt_dir, angle)

#dataset += ImageMaskDataset(image_dir, gt_dir, transforms.RandomVerticalFlip(p = 1.0))
#dataset += ImageMaskDataset(image_dir, gt_dir, transforms.RandomHorizontalFlip(p = 1.0))

In [ ]:
num_epochs = 10
learning_rate = 1e-3
batch_size = 5

data_len = len(dataset)
train_len = int(data_len * 0.9)
test_len = int(data_len * 0.1)

dataset_train, dataset_test = random_split(dataset, [train_len, test_len])
print(len(dataset_train), len(dataset_test))

dataloader_train = torch.utils.data.DataLoader(
    dataset_train,
    batch_size=batch_size,
    shuffle=True
)

dataloader_test = torch.utils.data.DataLoader(
    dataset_test,
    batch_size=batch_size,
    shuffle=True
)

import u_net_paper
from u_net_paper import UNet_paper

NUM_CHANNELS = 3
NUM_FILTERS = 32

import training

# Train the u-net model with the SGD optimizer
model = UNet_paper(NUM_CHANNELS, NUM_FILTERS).to(device)
criterion = losses.Dice()

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
training.train(model, criterion, dataloader_train, dataloader_test, optimizer, num_epochs)

#from torchsummary import summary
#summary(model, input_size=(NUM_CHANNELS, 400, 400))

In [ ]:
torch.save(model, "u_net.pkt")

In [ ]:
import image_mask_dataset
submission_dataloader = DataLoader(
    image_mask_dataset.FullSubmissionImageDataset(test_dir),
    batch_size=1
)

In [ ]:
model.eval()
toPIL = transforms.ToPILImage()

output_dir = "outputs/"

os.makedirs(output_dir)
for indexes, images in submission_dataloader:
    out = model(images.to(device)).view(2, 608, 608).cpu()
    toPIL(out[0]).save(output_dir + "file_{}.png".format(indexes.view(-1).item()))

In [ ]:
masks_to_submission("submission.csv", [output_dir + f in os.listdir(output_dir)])